In [1]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import numpy as np
import traceback
from keras.models import load_model
from string import ascii_uppercase
import mediapipe as mp
import pyttsx3  # for text-to-speech
import math

class Application:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Sign Language To Text Conversion")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("1300x700")

        # Initialize variables
        self.vs = cv2.VideoCapture(0)
        self.current_image = None
        self.current_image2 = None
        self.ccc = 0
        self.current_symbol = ""
        self.str = ""
        self.blank_flag = 0
        self.ct = {char: 0 for char in ascii_uppercase}
        self.prev_char = ''
        self.count = 0

        # Load model
        print("Loading model...")
        self.model = load_model(r'A:\desktop\progit\Sign-Language-To-Text-and-Speech-Conversion\fine_tune_models\fintune2.h5')
        print("Model loaded successfully.")

        self.setup_gui()
        self.speak_engine = pyttsx3.init()

        self.video_loop()

    def setup_gui(self):
        self.panel = tk.Label(self.root)
        self.panel.place(x=100, y=3, width=480, height=640)

        self.panel2 = tk.Label(self.root)
        self.panel2.place(x=700, y=115, width=400, height=400)

        self.T = tk.Label(self.root)
        self.T.place(x=60, y=5)
        self.T.config(text="Sign Language To Text Conversion", font=("Courier", 30, "bold"))

        self.panel3 = tk.Label(self.root)  # Current Symbol
        self.panel3.place(x=280, y=585)

        self.T1 = tk.Label(self.root)
        self.T1.place(x=10, y=580)
        self.T1.config(text="Character :", font=("Courier", 30, "bold"))

        self.panel5 = tk.Label(self.root)  # Sentence
        self.panel5.place(x=260, y=632)

        self.T3 = tk.Label(self.root)
        self.T3.place(x=10, y=632)
        self.T3.config(text="Sentence :", font=("Courier", 30, "bold"))

        self.T4 = tk.Label(self.root)
        self.T4.place(x=10, y=700)
        self.T4.config(text="Suggestions :", fg="red", font=("Courier", 30, "bold"))

        self.b1 = tk.Button(self.root, command=self.action1)
        self.b1.place(x=390, y=700)

        self.b2 = tk.Button(self.root, command=self.action2)
        self.b2.place(x=590, y=700)

        self.b3 = tk.Button(self.root, command=self.action3)
        self.b3.place(x=790, y=700)

        self.b4 = tk.Button(self.root, command=self.action4)
        self.b4.place(x=990, y=700)

        self.speak = tk.Button(self.root, text="Speak", font=("Courier", 20), wraplength=100, command=self.speak_fun)
        self.speak.place(x=1305, y=630)

        self.clear = tk.Button(self.root, text="Clear", font=("Courier", 20), wraplength=100, command=self.clear_fun)
        self.clear.place(x=1205, y=630)

    def destructor(self):
        print("Closing Application")
        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows()

    def video_loop(self):
        mp_hands = mp.solutions.hands
        hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
        mp_drawing = mp.solutions.drawing_utils
    
        try:
            ret, frame = self.vs.read()
            if not ret:
                print("Failed to grab frame")
                self.root.after(10, self.video_loop)
                return
    
            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = hands.process(frame_rgb)
    
            if result.multi_hand_landmarks:
                for hand_landmarks in result.multi_hand_landmarks:
                    x_min, y_min = float('inf'), float('inf')
                    x_max, y_max = float('-inf'), float('-inf')
    
                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        x_min, y_min = min(x_min, x), min(y_min, y)
                        x_max, y_max = max(x_max, x), max(y_max, y)
    
                    bbox = (x_min, y_min, x_max - x_min, y_max - y_min)
                    image = frame[y_min:y_max, x_min:x_max]
                    white = np.ones((400, 400, 3), np.uint8) * 255
    
                    h, w, _ = image.shape
                    os_x = (400 - w) // 2
                    os_y = (400 - h) // 2
    
                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        cv2.circle(white, (x - x_min + os_x, y - y_min + os_y), 5, (0, 0, 255), -1)
    
                    mp_drawing.draw_landmarks(white, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
                    # Resize image to 128x128
                    white_resized = cv2.resize(white, (128, 128))
                    white_expanded = np.expand_dims(white_resized, axis=0)
                    white_expanded = np.array(white_expanded, dtype='float32')
    
                    self.predict(white_expanded)
    
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            self.panel.imgtk = imgtk
            self.panel.config(image=imgtk)
    
        except Exception as e:
            print(f"An error occurred: {e}")
            traceback.print_exc()
    
        self.root.after(10, self.video_loop)

    def distance(p1, p2):
        return np.linalg.norm(np.array(p1) - np.array(p2))

    def predict(self, test_image):
        white = test_image
        prob = np.array(self.model.predict(white)[0], dtype='float32')
        ch1 = np.argmax(prob, axis=0)
        prob[ch1] = 0
        ch2 = np.argmax(prob, axis=0)
        prob[ch2] = 0
        ch3 = np.argmax(prob, axis=0)
        prob[ch3] = 0

        # Extracting the points from hand_landmarks
        pts = [(lm.x, lm.y) for lm in test_image.landmark]
    
        pl = [ch1, ch2]
    
        # condition for [Aemnst]
        l = [[5, 2], [5, 3], [3, 5], [3, 6], [3, 0], [3, 2], [6, 4], [6, 1], [6, 2], [6, 6], [6, 7], [6, 0], [6, 5], [4, 1],
             [1, 0], [1, 1], [6, 3], [1, 6], [5, 6], [5, 1], [4, 5], [1, 4], [1, 5], [2, 0], [2, 6], [4, 6], [1, 0], [5, 7],
             [1, 6], [6, 1], [7, 6], [2, 5], [7, 1], [5, 4], [7, 0], [7, 5], [7, 2]]
        if pl in l:
            if (pts[6][1] < pts[8][1] and pts[10][1] < pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] < pts[20][1]):
                ch1 = 0
                # print("00000")
    
        # condition for [o][s]
        l = [[2, 2], [2, 1]]
        if pl in l:
            if (pts[5][0] < pts[4][0]):
                ch1 = 0
                print("++++++++++++++++++")
                # print("00000")
    
        # condition for [c0][aemnst]
        l = [[0, 0], [0, 6], [0, 2], [0, 5], [0, 1], [0, 7], [5, 2], [7, 6], [7, 1]]
        pl = [ch1, ch2]
        if pl in l:
            if (pts[0][0] > pts[8][0] and pts[0][0] > pts[4][0] and pts[0][0] > pts[12][0] and pts[0][0] > pts[16][0] and pts[0][0] > pts[20][0]) and pts[5][0] > pts[4][0]:
                ch1 = 2
                # print("22222")
    
        # condition for [c0][aemnst]
        l = [[6, 0], [6, 6], [6, 2]]
        pl = [ch1, ch2]
        if pl in l:
            if distance(pts[8], pts[16]) < 52:
                ch1 = 2
                # print("22222")
    
        ##print(pts[2][1]+15>pts[16][1])
        # condition for [gh][bdfikruvw]
        l = [[1, 4], [1, 5], [1, 6], [1, 3], [1, 0]]
        pl = [ch1, ch2]
        if pl in l:
            if pts[6][1] > pts[8][1] and pts[14][1] < pts[16][1] and pts[18][1] < pts[20][1] and pts[0][0] < pts[8][0] and pts[0][0] < pts[12][0] and pts[0][0] < pts[16][0] and pts[0][0] < pts[20][0]:
                ch1 = 3
                print("33333c")
    
        # con for [gh][l]
        l = [[4, 6], [4, 1], [4, 5], [4, 3], [4, 7]]
        pl = [ch1, ch2]
        if pl in l:
            if pts[4][0] > pts[0][0]:
                ch1 = 3
                print("33333b")
    
        # con for [gh][pqz]
        l = [[5, 3], [5, 0], [5, 7], [5, 4], [5, 2], [5, 1], [5, 5]]
        pl = [ch1, ch2]
        if pl in l:
            if pts[2][1] + 15 < pts[16][1]:
                ch1 = 3
                print("33333a")
    
        # con for [l][x]
        l = [[6, 4], [6, 1], [6, 2]]
        pl = [ch1, ch2]
        if pl in l:
            if distance(pts[4], pts[11]) > 55:
                ch1 = 4
                # print("44444")
    
        # con for [l][d]
        l = [[1, 4], [1, 6], [1, 1]]
        pl = [ch1, ch2]
        if pl in l:
            if (distance(pts[4], pts[11]) > 50) and (pts[6][1] > pts[8][1] and pts[10][1] < pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] < pts[20][1]):
                ch1 = 4
                # print("44444")
    
        # con for [l][gh]
        l = [[3, 6], [3, 4]]
        pl = [ch1, ch2]
        if pl in l:
            if (pts[4][0] < pts[0][0]):
                ch1 = 4
                # print("44444")
    
        # con for [l][c0]
        l = [[2, 2], [2, 5], [2, 4]]
        pl = [ch1, ch2]
        if pl in l:
            if (pts[1][0] < pts[12][0]):
                ch1 = 4
                # print("44444")
    
        # con for [gh][z]
        l = [[3, 6], [3, 5], [3, 4]]
        pl = [ch1, ch2]
        if pl in l:
            if (pts[6][1] > pts[8][1] and pts[10][1] < pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] < pts[20][1]) and pts[4][1] > pts[10][1]:
                ch1 = 5
                print("55555b")
    
        # con for [gh][pq]
        l = [[3, 2], [3, 1], [3, 6]]
        pl = [ch1, ch2]
        if pl in l:
            if pts[4][1] + 17 > pts[8][1] and pts[4][1] + 17 > pts[12][1] and pts[4][1] + 17 > pts[16][1] and pts[4][1] + 17 > pts[20][1]:
                ch1 = 5
                print("55555a")
    
        # con for [l][pqz]
        l = [[4, 4], [4, 5], [4, 2], [7, 5], [7, 6], [7, 0]]
        pl = [ch1, ch2]
        if pl in l:
            if pts[4][0] > pts[0][0]:
                ch1 = 5
                # print("55555")
    
        # con for [pqz][aemnst]
        l = [[0, 2], [0, 6], [0, 1], [0, 5], [0, 0], [0, 7], [0, 4], [0, 3], [2, 7]]
        pl = [ch1, ch2]
        if pl in l:
            if pts[0][0] < pts[8][0] and pts[0][0] < pts[12][0] and pts[0][0] < pts[16][0] and pts[0][0] < pts[20][0]:
                ch1 = 6
                # print("66666")
    
        # con for [l][aemnst]
        l = [[0, 2], [0, 1], [0, 4], [0, 5]]
        pl = [ch1, ch2]
        if pl in l:
            if pts[4][0] < pts[0][0]:
                ch1 = 6
                # print("66666")
    
        return ch1
        print(f"Predicted classes: {ch1}, {ch2}, {ch3}")

        # Update current symbol and sentence
        self.current_symbol = ascii_uppercase[ch1]
        self.panel3.config(text=self.current_symbol, font=("Courier", 30, "bold"))

        self.str += self.current_symbol
        self.panel5.config(text=self.str, font=("Courier", 30, "bold"))

    def action1(self):
        self.str += self.word1.upper()

    def action2(self):
        self.str += self.word2.upper()

    def action3(self):
        self.str += self.word3.upper()

    def action4(self):
        self.str += self.word4.upper()

    def speak_fun(self):
        self.speak_engine.say(self.str)
        self.speak_engine.runAndWait()

    def clear_fun(self):
        self.str = ""
        self.word1 = ""
        self.word2 = ""
        self.word3 = ""
        self.word4 = ""
        self.panel5.config(text=self.str, font=("Courier", 30, "bold"))

if __name__ == "__main__":
    print("Starting Application...")
    app = Application()
    app.root.mainloop()


Starting Application...
Loading model...
Model loaded successfully.
1/1 [==============================] - 1s 1s/step
An error occurred: name 'hand_landmarks' is not defined
1/1 [==============================] - 0s 30ms/step
An error occurred: name 'hand_landmarks' is not defined


Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_28928\4067759497.py", line 139, in video_loop
    self.predict(white_expanded)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_28928\4067759497.py", line 167, in predict
    pts = [(lm.x, lm.y) for lm in hand_landmarks.landmark]
NameError: name 'hand_landmarks' is not defined
Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_28928\4067759497.py", line 139, in video_loop
    self.predict(white_expanded)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_28928\4067759497.py", line 167, in predict
    pts = [(lm.x, lm.y) for lm in hand_landmarks.landmark]
NameError: name 'hand_landmarks' is not defined


1/1 [==============================] - 0s 16ms/step
An error occurred: name 'hand_landmarks' is not defined
1/1 [==============================] - 0s 36ms/step
An error occurred: name 'hand_landmarks' is not defined


Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_28928\4067759497.py", line 139, in video_loop
    self.predict(white_expanded)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_28928\4067759497.py", line 167, in predict
    pts = [(lm.x, lm.y) for lm in hand_landmarks.landmark]
NameError: name 'hand_landmarks' is not defined
Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_28928\4067759497.py", line 139, in video_loop
    self.predict(white_expanded)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_28928\4067759497.py", line 167, in predict
    pts = [(lm.x, lm.y) for lm in hand_landmarks.landmark]
NameError: name 'hand_landmarks' is not defined


1/1 [==============================] - 0s 16ms/step
An error occurred: name 'hand_landmarks' is not defined
1/1 [==============================] - 0s 30ms/step
An error occurred: name 'hand_landmarks' is not defined


...

1/1 [==============================] - 0s 38ms/step
An error occurred: name 'hand_landmarks' is not defined


Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_28928\4067759497.py", line 139, in video_loop
    self.predict(white_expanded)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_28928\4067759497.py", line 167, in predict
    pts = [(lm.x, lm.y) for lm in hand_landmarks.landmark]
NameError: name 'hand_landmarks' is not defined


1/1 [==============================] - 1s 527ms/step
An error occurred: name 'hand_landmarks' is not defined


Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_28928\4067759497.py", line 139, in video_loop
    self.predict(white_expanded)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_28928\4067759497.py", line 167, in predict
    pts = [(lm.x, lm.y) for lm in hand_landmarks.landmark]
NameError: name 'hand_landmarks' is not defined


Closing Application


In [ ]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import numpy as np
import traceback
from keras.models import load_model
from string import ascii_uppercase
import mediapipe as mp
import pyttsx3  # for text-to-speech

class Application:
    dicttt = {}
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Sign Language To Text Conversion")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("1300x700")

        # Initialize variables
        self.vs = cv2.VideoCapture(0)
        self.current_image = None
        self.current_image2 = None
        self.ccc = 0
        self.current_symbol = ""
        self.str = ""
        self.blank_flag = 0
        self.ct = {char: 0 for char in ascii_uppercase}
        self.prev_char = ''
        self.count = 0

        # Load model
        print("Loading model...")
        self.model = load_model(r'A:\desktop\progit\Sign-Language-To-Text-and-Speech-Conversion\fine_tune_models\fintune2.h5')
        print("Model loaded successfully.")

        self.setup_gui()
        self.speak_engine = pyttsx3.init()

        self.video_loop()

    def setup_gui(self):
        self.panel = tk.Label(self.root)
        self.panel.place(x=100, y=3, width=480, height=640)

        self.panel2 = tk.Label(self.root)
        self.panel2.place(x=700, y=115, width=400, height=400)

        self.T = tk.Label(self.root)
        self.T.place(x=60, y=5)
        self.T.config(text="Sign Language To Text Conversion", font=("Courier", 30, "bold"))

        self.panel3 = tk.Label(self.root)  # Current Symbol
        self.panel3.place(x=280, y=585)

        self.T1 = tk.Label(self.root)
        self.T1.place(x=10, y=580)
        self.T1.config(text="Character :", font=("Courier", 30, "bold"))

        self.panel5 = tk.Label(self.root)  # Sentence
        self.panel5.place(x=260, y=632)

        self.T3 = tk.Label(self.root)
        self.T3.place(x=10, y=632)
        self.T3.config(text="Sentence :", font=("Courier", 30, "bold"))

        self.T4 = tk.Label(self.root)
        self.T4.place(x=10, y=700)
        self.T4.config(text="Suggestions :", fg="red", font=("Courier", 30, "bold"))

        self.b1 = tk.Button(self.root, command=self.action1)
        self.b1.place(x=390, y=700)

        self.b2 = tk.Button(self.root, command=self.action2)
        self.b2.place(x=590, y=700)

        self.b3 = tk.Button(self.root, command=self.action3)
        self.b3.place(x=790, y=700)

        self.b4 = tk.Button(self.root, command=self.action4)
        self.b4.place(x=990, y=700)

        self.speak = tk.Button(self.root, text="Speak", font=("Courier", 20), wraplength=100, command=self.speak_fun)
        self.speak.place(x=1305, y=630)

        self.clear = tk.Button(self.root, text="Clear", font=("Courier", 20), wraplength=100, command=self.clear_fun)
        self.clear.place(x=1205, y=630)

    def destructor(self):
        print("Closing Application")
        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows()

    def video_loop(self):
        mp_hands = mp.solutions.hands
        hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
        mp_drawing = mp.solutions.drawing_utils
    
        try:
            ret, frame = self.vs.read()
            if not ret:
                print("Failed to grab frame")
                self.root.after(10, self.video_loop)
                return
    
            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = hands.process(frame_rgb)
    
            if result.multi_hand_landmarks:
                for hand_landmarks in result.multi_hand_landmarks:
                    x_min, y_min = float('inf'), float('inf')
                    x_max, y_max = float('-inf'), float('-inf')
    
                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        x_min, y_min = min(x_min, x), min(y_min, y)
                        x_max, y_max = max(x_max, x), max(y_max, y)
    
                    bbox = (x_min, y_min, x_max - x_min, y_max - y_min)
                    image = frame[y_min:y_max, x_min:x_max]
                    white = np.ones((400, 400, 3), np.uint8) * 255
    
                    h, w, _ = image.shape
                    os_x = (400 - w) // 2
                    os_y = (400 - h) // 2
    
                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        cv2.circle(white, (x - x_min + os_x, y - y_min + os_y), 5, (0, 0, 255), -1)
    
                    mp_drawing.draw_landmarks(white, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
                    # Resize image to 128x128
                    white_resized = cv2.resize(white, (128, 128))
                    white_expanded = np.expand_dims(white_resized, axis=0)
                    white_expanded = np.array(white_expanded, dtype='float32')
    
                    self.predict(white_expanded, hand_landmarks.landmark)
    
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            self.panel.imgtk = imgtk
            self.panel.config(image=imgtk)
    
        except Exception as e:
            print(f"An error occurred: {e}")
            traceback.print_exc()
    
        self.root.after(10, self.video_loop)

    def distance(self, p1, p2):
        return np.linalg.norm(np.array(p1) - np.array(p2))

    def predict(self, test_image, landmarks):
        kok = []
        prob = np.array(self.model.predict(test_image)[0], dtype='float32')
        ch1 = np.argmax(prob, axis=0)
        prob[ch1] = 0
        ch2 = np.argmax(prob, axis=0)
        prob[ch2] = 0
        ch3 = np.argmax(prob, axis=0)
        prob[ch3] = 0

        # Extracting the points from hand_landmarks
        pts = [(lm.x, lm.y) for lm in landmarks]
    
        pl = [ch1, ch2]
    
        # condition for [Aemnst]
        l = [[5, 2], [5, 3], [3, 5], [3, 6], [3, 0], [3, 2], [6, 4], [6, 1], [6, 2], [6, 6], [6, 7], [6, 0], [6, 5], [4, 1],
             [1, 0], [1, 1], [6, 3], [1, 6], [5, 6], [5, 1], [4, 5], [1, 4], [1, 5], [2, 0], [2, 6], [4, 6], [1, 0], [5, 7],
             [1, 6], [6, 1], [7, 6], [2, 5], [7, 1], [5, 4], [7, 0], [7, 5], [7, 2]]
        if pl in l:
            if (pts[6][1] < pts[8][1] and pts[10][1] < pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] < pts[20][1]):
                ch1 = 0
    
        # condition for [o][s]
        l = [[2, 2], [2, 1]]
        if pl in l:
            if (pts[5][0] < pts[4][0]):
                ch1 = 0
    
        # condition for [f]
        l = [[3, 3], [3, 1], [3, 0]]
        if pl in l:
            if (pts[12][1] < pts[10][1]):
                ch1 = 0
    
        # condition for [d]
        l = [[3, 3], [3, 4]]
        if pl in l:
            if (pts[8][1] < pts[6][1]):
                ch1 = 0

        # -------------------------condn for subgroups starts
        #
        if ch1 == 0:
            ch1='S'
            if pts[4][0] < pts[6][0] and pts[4][0] < pts[10][0] and pts[4][0] < pts[14][0] and pts[4][0] < pts[18][0]:
                ch1 = 'A'
            if pts[4][0] > pts[6][0] and pts[4][0] < pts[10][0] and pts[4][0] < pts[14][0] and pts[4][0] < pts[18][0] and pts[4][1] < pts[14][1] and pts[4][1] < pts[18][1] :
                ch1 = 'T'
            if pts[4][1] > pts[8][1] and pts[4][1] > pts[12][1] and pts[4][1] > pts[16][1] and pts[4][1] > pts[20][1]:
                ch1 = 'E'
            if pts[4][0] > pts[6][0] and pts[4][0] > pts[10][0] and pts[4][0] > pts[14][0] and  pts[4][1] < pts[18][1]:
                ch1 = 'M'
            if pts[4][0] > pts[6][0] and pts[4][0] > pts[10][0]  and  pts[4][1] < pts[18][1] and pts[4][1] < pts[14][1]:
                ch1 = 'N'
    
        if ch1 == 2:
            if distance(pts[12], pts[4]) > 42:
                ch1 = 'C'
            else:
                ch1 = 'O'
    
        if ch1 == 3:
            if (distance(pts[8], pts[12])) > 72:
                ch1 = 'G'
            else:
                ch1 = 'H'
    
        if ch1 == 7:
            if distance(pts[8], pts[4]) > 42:
                ch1 = 'Y'
            else:
                ch1 = 'J'
    
        if ch1 == 4:
            ch1 = 'L'
    
        if ch1 == 6:
            ch1 = 'X'
    
        if ch1 == 5:
            if pts[4][0] > pts[12][0] and pts[4][0] > pts[16][0] and pts[4][0] > pts[20][0]:
                if pts[8][1] < pts[5][1]:
                    ch1 = 'Z'
                else:
                    ch1 = 'Q'
            else:
                ch1 = 'P'
    
        if ch1 == 1:
            if (pts[6][1] > pts[8][1] and pts[10][1] > pts[12][1] and pts[14][1] > pts[16][1] and pts[18][1] >pts[20][1]):
                ch1 = 'B'
            if (pts[6][1] > pts[8][1] and pts[10][1] < pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] <pts[20][1]):
                ch1 = 'D'
            if (pts[6][1] < pts[8][1] and pts[10][1] > pts[12][1] and pts[14][1] > pts[16][1] and pts[18][1] > pts[20][1]):
                ch1 = 'F'
            if (pts[6][1] < pts[8][1] and pts[10][1] < pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] > pts[20][1]):
                ch1 = 'I'
            if (pts[6][1] > pts[8][1] and pts[10][1] > pts[12][1] and pts[14][1] > pts[16][1] and pts[18][1] < pts[20][1]):
                ch1 = 'W'
            if  (pts[6][1] > pts[8][1] and pts[10][1] > pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] < pts[20][1]) and pts[4][1]<pts[9][1]:
                ch1 = 'K'
            if ((distance(pts[8], pts[12]) - distance(pts[6], pts[10])) < 8) and (pts[6][1] > pts[8][1] and pts[10][1] > pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] < pts[20][1]):
                ch1 = 'U'
            if ((distance(pts[8], pts[12]) - distance(pts[6], pts[10])) >= 8) and (pts[6][1] > pts[8][1] and pts[10][1] > pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] < pts[20][1]) and (pts[4][1] >pts[9][1]):
                ch1 = 'V'
    
            if (pts[8][0] > pts[12][0]) and (pts[6][1] > pts[8][1] and pts[10][1] > pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] < pts[20][1]):
                ch1 = 'R'
    
        if ch1== 1 or 'E' or 'S' or 'X' or 'Y' or 'B':
            if (pts[6][1] > pts[8][1] and pts[10][1] < pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] > pts[20][1]):
                ch1 = 'Space'
    
        if ch1== 'E' or 'Y' or 'B':
            if (pts[4][0] < pts[5][0] ):
                ch1 = 'Next'

        if ch1== 'Next' or 'B' or 'C' or 'H' or 'F':
            if (pts[0][0] > pts[8][0] and pts[0][0] > pts[12][0] and pts[0][0] > pts[16][0] and pts[0][0] > pts[20][0]) and pts[4][1]<pts[8][1] and pts[4][1]<pts[12][1] and pts[4][1]<pts[16][1] and pts[4][1]<pts[20][1]:
                ch1 = 'Backspace'

        print("ch1=", ch1, " ch2=", ch2, " ch3=", ch3)
        kok.append(ch1)

                # # [0->aemnst][1->bfdiuvwkr][2->co][3->gh][4->l][5->pqz][6->x][7->yj]
        if ch1 != 1:
            if (ch1,ch2) in dicttt:
                dicttt[(ch1,ch2)] += 1
            else:
                dicttt[(ch1,ch2)] = 1

        frame = cv2.putText(frame, "Predicted " + str(ch1), (30, 80),
                                    cv2.FONT_HERSHEY_SIMPLEX,
                                    3, (0, 0, 255), 2, cv2.LINE_AA)

    
        if self.blank_flag == 0 and ch1 == 0:
            self.blank_flag = 1
        else:
            self.blank_flag = 0
            self.current_symbol = ascii_uppercase[ch1]
            self.ct[self.current_symbol] += 1
    
            if self.ct[self.current_symbol] > 20:
                self.ct = {char: 0 for char in ascii_uppercase}
                if self.current_symbol == self.prev_char:
                    self.count += 1
                else:
                    self.count = 0
                self.prev_char = self.current_symbol
    
                if self.count == 2:
                    self.count = 0
                    if self.current_symbol == 'Z':
                        self.str += " "
                    else:
                        self.str += self.current_symbol
        self.panel3.config(text=self.current_symbol, font=("Courier", 30, "bold"))
        self.panel5.config(text=self.str, font=("Courier", 30, "bold"))

    def action1(self):
        print("action1")

    def action2(self):
        print("action2")

    def action3(self):
        print("action3")

    def action4(self):
        print("action4")

    def speak_fun(self):
        text = self.str.strip()
        if text:
            self.speak_engine.say(text)
            self.speak_engine.runAndWait()

    def clear_fun(self):
        self.str = ""
        self.panel5.config(text=self.str, font=("Courier", 30, "bold"))

# Main
if __name__ == "__main__":
    print("Starting Application...")
    pba = Application()
    pba.root.mainloop()


Starting Application...
Loading model...
Model loaded successfully.
1/1 [==============================] - 0s 70ms/step
ch1= Next  ch2= 21  ch3= 0
An error occurred: name 'dicttt' is not defined
1/1 [==============================] - 0s 16ms/step
ch1= Next  ch2= 0  ch3= 0
An error occurred: name 'dicttt' is not defined


Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_9688\3935575609.py", line 139, in video_loop
    self.predict(white_expanded, hand_landmarks.landmark)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_9688\3935575609.py", line 283, in predict
    if (ch1,ch2) in dicttt:
NameError: name 'dicttt' is not defined. Did you mean: 'dict'?
Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_9688\3935575609.py", line 139, in video_loop
    self.predict(white_expanded, hand_landmarks.landmark)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_9688\3935575609.py", line 283, in predict
    if (ch1,ch2) in dicttt:
NameError: name 'dicttt' is not defined. Did you mean: 'dict'?


1/1 [==============================] - 0s 22ms/step
ch1= Next  ch2= 6  ch3= 0
An error occurred: name 'dicttt' is not defined


Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_9688\3935575609.py", line 139, in video_loop
    self.predict(white_expanded, hand_landmarks.landmark)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_9688\3935575609.py", line 283, in predict
    if (ch1,ch2) in dicttt:
NameError: name 'dicttt' is not defined. Did you mean: 'dict'?


1/1 [==============================] - 0s 32ms/step
ch1= Next  ch2= 21  ch3= 0
An error occurred: name 'dicttt' is not defined


...

Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_9688\3935575609.py", line 139, in video_loop
    self.predict(white_expanded, hand_landmarks.landmark)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_9688\3935575609.py", line 283, in predict
    if (ch1,ch2) in dicttt:
NameError: name 'dicttt' is not defined. Did you mean: 'dict'?


1/1 [==============================] - 0s 25ms/step
ch1= P  ch2= 0  ch3= 0
An error occurred: name 'dicttt' is not defined
1/1 [==============================] - 0s 21ms/step
ch1= Backspace  ch2= 10  ch3= 0
An error occurred: name 'dicttt' is not defined


Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_9688\3935575609.py", line 139, in video_loop
    self.predict(white_expanded, hand_landmarks.landmark)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_9688\3935575609.py", line 283, in predict
    if (ch1,ch2) in dicttt:
NameError: name 'dicttt' is not defined. Did you mean: 'dict'?
Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_9688\3935575609.py", line 139, in video_loop
    self.predict(white_expanded, hand_landmarks.landmark)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_9688\3935575609.py", line 283, in predict
    if (ch1,ch2) in dicttt:
NameError: name 'dicttt' is not defined. Did you mean: 'dict'?


1/1 [==============================] - 0s 18ms/step
ch1= Next  ch2= 15  ch3= 0
An error occurred: name 'dicttt' is not defined


Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_9688\3935575609.py", line 139, in video_loop
    self.predict(white_expanded, hand_landmarks.landmark)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_9688\3935575609.py", line 283, in predict
    if (ch1,ch2) in dicttt:
NameError: name 'dicttt' is not defined. Did you mean: 'dict'?


In [1]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import numpy as np
import traceback
from keras.models import load_model
from string import ascii_uppercase
import mediapipe as mp
import pyttsx3  # for text-to-speech

class Application:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Sign Language To Text Conversion")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("1300x700")

        # Initialize variables
        self.vs = cv2.VideoCapture(0)
        self.current_image = None
        self.current_image2 = None
        self.ccc = 0
        self.current_symbol = ""
        self.str = ""
        self.blank_flag = 0
        self.ct = {char: 0 for char in ascii_uppercase}
        self.prev_char = ''
        self.count = 0
        self.dicttt = {}

        # Load model
        print("Loading model...")
        self.model = load_model(r'A:\desktop\progit\Sign-Language-To-Text-and-Speech-Conversion\fine_tune_models\fintune2.h5')
        print("Model loaded successfully.")

        self.setup_gui()
        self.speak_engine = pyttsx3.init()

        self.video_loop()

    def setup_gui(self):
        self.panel = tk.Label(self.root)
        self.panel.place(x=100, y=3, width=480, height=640)

        self.panel2 = tk.Label(self.root)
        self.panel2.place(x=700, y=115, width=400, height=400)

        self.T = tk.Label(self.root)
        self.T.place(x=60, y=5)
        self.T.config(text="Sign Language To Text Conversion", font=("Courier", 30, "bold"))

        self.panel3 = tk.Label(self.root)  # Current Symbol
        self.panel3.place(x=280, y=585)

        self.T1 = tk.Label(self.root)
        self.T1.place(x=10, y=580)
        self.T1.config(text="Character :", font=("Courier", 30, "bold"))

        self.panel5 = tk.Label(self.root)  # Sentence
        self.panel5.place(x=260, y=632)

        self.T3 = tk.Label(self.root)
        self.T3.place(x=10, y=632)
        self.T3.config(text="Sentence :", font=("Courier", 30, "bold"))

        self.T4 = tk.Label(self.root)
        self.T4.place(x=10, y=700)
        self.T4.config(text="Suggestions :", fg="red", font=("Courier", 30, "bold"))

        self.b1 = tk.Button(self.root, command=self.action1)
        self.b1.place(x=390, y=700)

        self.b2 = tk.Button(self.root, command=self.action2)
        self.b2.place(x=590, y=700)

        self.b3 = tk.Button(self.root, command=self.action3)
        self.b3.place(x=790, y=700)

        self.b4 = tk.Button(self.root, command=self.action4)
        self.b4.place(x=990, y=700)

        self.speak = tk.Button(self.root, text="Speak", font=("Courier", 20), wraplength=100, command=self.speak_fun)
        self.speak.place(x=1305, y=630)

        self.clear = tk.Button(self.root, text="Clear", font=("Courier", 20), wraplength=100, command=self.clear_fun)
        self.clear.place(x=1205, y=630)

    def destructor(self):
        print("Closing Application")
        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows()

    def video_loop(self):
        mp_hands = mp.solutions.hands
        hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
        mp_drawing = mp.solutions.drawing_utils

        try:
            ret, frame = self.vs.read()
            if not ret:
                print("Failed to grab frame")
                self.root.after(10, self.video_loop)
                return

            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = hands.process(frame_rgb)

            if result.multi_hand_landmarks:
                for hand_landmarks in result.multi_hand_landmarks:
                    x_min, y_min = float('inf'), float('inf')
                    x_max, y_max = float('-inf'), float('-inf')

                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        x_min, y_min = min(x_min, x), min(y_min, y)
                        x_max, y_max = max(x_max, x), max(y_max, y)

                    bbox = (x_min, y_min, x_max - x_min, y_max - y_min)
                    image = frame[y_min:y_max, x_min:x_max]
                    white = np.ones((400, 400, 3), np.uint8) * 255

                    h, w, _ = image.shape
                    os_x = (400 - w) // 2
                    os_y = (400 - h) // 2

                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        cv2.circle(white, (x - x_min + os_x, y - y_min + os_y), 5, (0, 0, 255), -1)

                    mp_drawing.draw_landmarks(white, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    # Resize image to 128x128
                    white_resized = cv2.resize(white, (128, 128))
                    white_expanded = np.expand_dims(white_resized, axis=0)
                    white_expanded = np.array(white_expanded, dtype='float32')

                    self.predict(white_expanded, hand_landmarks.landmark)

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            self.panel.imgtk = imgtk
            self.panel.config(image=imgtk)

        except Exception as e:
            print(f"An error occurred: {e}")
            traceback.print_exc()

        self.root.after(10, self.video_loop)

    def distance(self, p1, p2):
        return np.linalg.norm(np.array(p1) - np.array(p2))

    def predict(self, test_image, landmarks):
        prob = np.array(self.model.predict(test_image)[0], dtype='float32')
        ch1 = np.argmax(prob, axis=0)
        prob[ch1] = 0
        ch2 = np.argmax(prob, axis=0)
        prob[ch2] = 0
        ch3 = np.argmax(prob, axis=0)
        prob[ch3] = 0

        pts = [(lm.x, lm.y) for lm in landmarks]
    
        pl = [ch1, ch2]

        # Example condition for adjustments (these need to be defined based on actual requirements)
        if pl == [5, 2] and pts[6][1] < pts[8][1]:
            ch1 = 0

        # Example for other specific conditions
        if pl == [2, 2] and pts[5][0] < pts[4][0]:
            ch1 = 0

        # Add more conditions as needed...
        
        # Map integer predictions to letters
        letter_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H'}
        ch1 = letter_mapping.get(ch1, 'Unknown')

        print(f"Predicted characters: ch1={ch1}, ch2={ch2}, ch3={ch3}")

        if self.blank_flag == 0 and ch1 == 0:
            self.blank_flag = 1
        else:
            self.blank_flag = 0
            self.current_symbol = ch1
            self.ct[self.current_symbol] += 1
    
            if self.ct[self.current_symbol] > 20:
                self.ct = {char: 0 for char in ascii_uppercase}
                if self.current_symbol == self.prev_char:
                    self.count += 1
                else:
                    self.count = 0
                self.prev_char = self.current_symbol
    
                if self.count == 2:
                    self.count = 0
                    if self.current_symbol == 'Z':
                        self.str += " "
                    else:
                        self.str += self.current_symbol
        self.panel3.config(text=self.current_symbol, font=("Courier", 30, "bold"))
        self.panel5.config(text=self.str, font=("Courier", 30, "bold"))

    def action1(self):
        print("action1")

    def action2(self):
        print("action2")

    def action3(self):
        print("action3")

    def action4(self):
        print("action4")

    def speak_fun(self):
        text = self.str.strip()
        if text:
            self.speak_engine.say(text)
            self.speak_engine.runAndWait()

    def clear_fun(self):
        self.str = ""
        self.panel5.config(text=self.str, font=("Courier", 30, "bold"))

print("Starting Application...")
app = Application()
app.root.mainloop()


Starting Application...
Loading model...
Model loaded successfully.
1/1 [==============================] - 1s 1s/step
Predicted characters: ch1=B, ch2=0, ch3=0
1/1 [==============================] - 0s 47ms/step
Predicted characters: ch1=B, ch2=0, ch3=0
1/1 [==============================] - 0s 31ms/step
Predicted characters: ch1=B, ch2=0, ch3=0
1/1 [==============================] - 0s 31ms/step
Predicted characters: ch1=B, ch2=0, ch3=0
1/1 [==============================] - 0s 31ms/step
Predicted characters: ch1=B, ch2=0, ch3=0
1/1 [==============================] - 0s 47ms/step
Predicted characters: ch1=B, ch2=0, ch3=0
1/1 [==============================] - 0s 47ms/step
Predicted characters: ch1=B, ch2=0, ch3=0
1/1 [==============================] - 0s 31ms/step
Predicted characters: ch1=B, ch2=0, ch3=0
1/1 [==============================] - 0s 32ms/step
Predicted characters: ch1=B, ch2=0, ch3=0
1/1 [==============================] - 0s 32ms/step
Predicted characters: ch1=B, ch2=0, 

Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_55572\1335448478.py", line 139, in video_loop
    self.predict(white_expanded, hand_landmarks.landmark)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_55572\1335448478.py", line 190, in predict
    self.ct[self.current_symbol] += 1
KeyError: 'Unknown'


1/1 [==============================] - 0s 32ms/step
Predicted characters: ch1=Unknown, ch2=0, ch3=0
An error occurred: 'Unknown'
1/1 [==============================] - 0s 16ms/step
Predicted characters: ch1=C, ch2=0, ch3=0


Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_55572\1335448478.py", line 139, in video_loop
    self.predict(white_expanded, hand_landmarks.landmark)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_55572\1335448478.py", line 190, in predict
    self.ct[self.current_symbol] += 1
KeyError: 'Unknown'


1/1 [==============================] - 0s 19ms/step
Predicted characters: ch1=A, ch2=11, ch3=23
1/1 [==============================] - 0s 31ms/step
Predicted characters: ch1=Unknown, ch2=8, ch3=0
An error occurred: 'Unknown'
1/1 [==============================] - 0s 32ms/step
Predicted characters: ch1=Unknown, ch2=8, ch3=0
An error occurred: 'Unknown'


...

1/1 [==============================] - 0s 16ms/step
Predicted characters: ch1=Unknown, ch2=0, ch3=0
An error occurred: 'Unknown'
1/1 [==============================] - 0s 16ms/step
Predicted characters: ch1=Unknown, ch2=0, ch3=0
An error occurred: 'Unknown'


Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_55572\1335448478.py", line 139, in video_loop
    self.predict(white_expanded, hand_landmarks.landmark)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_55572\1335448478.py", line 190, in predict
    self.ct[self.current_symbol] += 1
KeyError: 'Unknown'
Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_55572\1335448478.py", line 139, in video_loop
    self.predict(white_expanded, hand_landmarks.landmark)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_55572\1335448478.py", line 190, in predict
    self.ct[self.current_symbol] += 1
KeyError: 'Unknown'


1/1 [==============================] - 0s 31ms/step
Predicted characters: ch1=Unknown, ch2=0, ch3=0
An error occurred: 'Unknown'
1/1 [==============================] - 0s 16ms/step
Predicted characters: ch1=B, ch2=0, ch3=0


Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_55572\1335448478.py", line 139, in video_loop
    self.predict(white_expanded, hand_landmarks.landmark)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_55572\1335448478.py", line 190, in predict
    self.ct[self.current_symbol] += 1
KeyError: 'Unknown'


Closing Application


In [1]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import numpy as np
import traceback
from keras.models import load_model
from string import ascii_uppercase
import mediapipe as mp
import pyttsx3  # for text-to-speech

class Application:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Sign Language To Text Conversion")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("1300x700")

        # Initialize variables
        self.vs = cv2.VideoCapture(0)
        self.current_image = None
        self.current_image2 = None
        self.ccc = 0
        self.current_symbol = ""
        self.str = ""
        self.blank_flag = 0
        self.ct = {char: 0 for char in ascii_uppercase}
        self.prev_char = ''
        self.count = 0
        self.dicttt = {}

        # Load model
        print("Loading model...")
        self.model = load_model(r'A:\desktop\progit\Sign-Language-To-Text-and-Speech-Conversion\fine_tune_models\fintune2.h5')
        print("Model loaded successfully.")

        self.setup_gui()
        self.speak_engine = pyttsx3.init()

        self.video_loop()

    def setup_gui(self):
        self.panel = tk.Label(self.root)
        self.panel.place(x=100, y=3, width=480, height=640)

        self.panel2 = tk.Label(self.root)
        self.panel2.place(x=700, y=115, width=400, height=400)

        self.T = tk.Label(self.root)
        self.T.place(x=60, y=5)
        self.T.config(text="Sign Language To Text Conversion", font=("Courier", 30, "bold"))

        self.panel3 = tk.Label(self.root)  # Current Symbol
        self.panel3.place(x=280, y=585)

        self.T1 = tk.Label(self.root)
        self.T1.place(x=10, y=580)
        self.T1.config(text="Character :", font=("Courier", 30, "bold"))

        self.panel5 = tk.Label(self.root)  # Sentence
        self.panel5.place(x=260, y=632)

        self.T3 = tk.Label(self.root)
        self.T3.place(x=10, y=632)
        self.T3.config(text="Sentence :", font=("Courier", 30, "bold"))

        self.T4 = tk.Label(self.root)
        self.T4.place(x=10, y=700)
        self.T4.config(text="Suggestions :", fg="red", font=("Courier", 30, "bold"))

        self.b1 = tk.Button(self.root, command=self.action1)
        self.b1.place(x=390, y=700)

        self.b2 = tk.Button(self.root, command=self.action2)
        self.b2.place(x=590, y=700)

        self.b3 = tk.Button(self.root, command=self.action3)
        self.b3.place(x=790, y=700)

        self.b4 = tk.Button(self.root, command=self.action4)
        self.b4.place(x=990, y=700)

        self.speak = tk.Button(self.root, text="Speak", font=("Courier", 20), wraplength=100, command=self.speak_fun)
        self.speak.place(x=1305, y=630)

        self.clear = tk.Button(self.root, text="Clear", font=("Courier", 20), wraplength=100, command=self.clear_fun)
        self.clear.place(x=1205, y=630)

    def destructor(self):
        print("Closing Application")
        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows()

    def video_loop(self):
        mp_hands = mp.solutions.hands
        hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
        mp_drawing = mp.solutions.drawing_utils

        try:
            ret, frame = self.vs.read()
            if not ret:
                print("Failed to grab frame")
                self.root.after(10, self.video_loop)
                return

            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = hands.process(frame_rgb)

            if result.multi_hand_landmarks:
                for hand_landmarks in result.multi_hand_landmarks:
                    x_min, y_min = float('inf'), float('inf')
                    x_max, y_max = float('-inf'), float('-inf')

                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        x_min, y_min = min(x_min, x), min(y_min, y)
                        x_max, y_max = max(x_max, x), max(y_max, y)

                    bbox = (x_min, y_min, x_max - x_min, y_max - y_min)
                    image = frame[y_min:y_max, x_min:x_max]
                    white = np.ones((400, 400, 3), np.uint8) * 255

                    h, w, _ = image.shape
                    os_x = (400 - w) // 2
                    os_y = (400 - h) // 2

                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        cv2.circle(white, (x - x_min + os_x, y - y_min + os_y), 5, (0, 0, 255), -1)

                    mp_drawing.draw_landmarks(white, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    # Resize image to 128x128
                    white_resized = cv2.resize(white, (128, 128))
                    white_expanded = np.expand_dims(white_resized, axis=0)
                    white_expanded = np.array(white_expanded, dtype='float32')

                    self.predict(white_expanded, hand_landmarks.landmark)

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            self.panel.imgtk = imgtk
            self.panel.config(image=imgtk)

        except Exception as e:
            print(f"An error occurred: {e}")
            traceback.print_exc()

        self.root.after(10, self.video_loop)

    def draw_hand_lines(self, white, os_x, os_y, pts):
        for t in range(0, 4):
            cv2.line(white, (pts[t][0] + os_x, pts[t][1] + os_y), 
                     (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), (0, 255, 0), 3)
        for t in range(5, 8):
            cv2.line(white, (pts[t][0] + os_x, pts[t][1] + os_y), 
                     (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), (0, 255, 0), 3)
        for t in range(9, 12):
            cv2.line(white, (pts[t][0] + os_x, pts[t][1] + os_y), 
                     (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), (0, 255, 0), 3)
        for t in range(13, 16):
            cv2.line(white, (pts[t][0] + os_x, pts[t][1] + os_y), 
                     (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), (0, 255, 0), 3)
        for t in range(17, 20):
            cv2.line(white, (pts[t][0] + os_x, pts[t][1] + os_y), 
                     (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), (0, 255, 0), 3)
        for t in [0, 5, 9, 13, 17]:
            cv2.line(white, (pts[t][0] + os_x, pts[t][1] + os_y), 
                     (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), (0, 255, 0), 3)
            cv2.line(white, (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), 
                     (pts[t + 2][0] + os_x, pts[t + 2][1] + os_y), (0, 255, 0), 3)
        for t in range(1, 20, 4):
            cv2.line(white, (pts[0][0] + os_x, pts[0][1] + os_y), 
                     (pts[t][0] + os_x, pts[t][1] + os_y), (0, 255, 0), 3)


    def distance(self, p1, p2):
        return np.linalg.norm(np.array(p1) - np.array(p2))

    def predict(self, test_image, landmarks):
        kok = []
        prob = np.array(self.model.predict(test_image)[0], dtype='float32')
        ch1 = np.argmax(prob, axis=0)
        prob[ch1] = 0
        ch2 = np.argmax(prob, axis=0)
        prob[ch2] = 0
        ch3 = np.argmax(prob, axis=0)
        prob[ch3] = 0

        pts = [(lm.x, lm.y) for lm in landmarks]
    
        pl = [ch1, ch2]

        # Example condition for adjustments (these need to be defined based on actual requirements)
        if pl == [5, 2] and pts[6][1] < pts[8][1]:
            ch1 = 0

        # Example for other specific conditions
        if pl == [2, 2] and pts[5][0] < pts[4][0]:
            ch1 = 0

        # Extracting the points from hand_landmarks
        pts = [(lm.x, lm.y) for lm in landmarks]
    
        pl = [ch1, ch2]
    
        # condition for [Aemnst]
        l = [[5, 2], [5, 3], [3, 5], [3, 6], [3, 0], [3, 2], [6, 4], [6, 1], [6, 2], [6, 6], [6, 7], [6, 0], [6, 5], [4, 1],
             [1, 0], [1, 1], [6, 3], [1, 6], [5, 6], [5, 1], [4, 5], [1, 4], [1, 5], [2, 0], [2, 6], [4, 6], [1, 0], [5, 7],
             [1, 6], [6, 1], [7, 6], [2, 5], [7, 1], [5, 4], [7, 0], [7, 5], [7, 2]]
        if pl in l:
            if (pts[6][1] < pts[8][1] and pts[10][1] < pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] < pts[20][1]):
                ch1 = 0
    
        # condition for [o][s]
        l = [[2, 2], [2, 1]]
        if pl in l:
            if (pts[5][0] < pts[4][0]):
                ch1 = 0
    
        # condition for [f]
        l = [[3, 3], [3, 1], [3, 0]]
        if pl in l:
            if (pts[12][1] < pts[10][1]):
                ch1 = 0
    
        # condition for [d]
        l = [[3, 3], [3, 4]]
        if pl in l:
            if (pts[8][1] < pts[6][1]):
                ch1 = 0

        # -------------------------condn for subgroups starts
        #
        if ch1 == 0:
            ch1='S'
            if pts[4][0] < pts[6][0] and pts[4][0] < pts[10][0] and pts[4][0] < pts[14][0] and pts[4][0] < pts[18][0]:
                ch1 = 'A'
            if pts[4][0] > pts[6][0] and pts[4][0] < pts[10][0] and pts[4][0] < pts[14][0] and pts[4][0] < pts[18][0] and pts[4][1] < pts[14][1] and pts[4][1] < pts[18][1] :
                ch1 = 'T'
            if pts[4][1] > pts[8][1] and pts[4][1] > pts[12][1] and pts[4][1] > pts[16][1] and pts[4][1] > pts[20][1]:
                ch1 = 'E'
            if pts[4][0] > pts[6][0] and pts[4][0] > pts[10][0] and pts[4][0] > pts[14][0] and  pts[4][1] < pts[18][1]:
                ch1 = 'M'
            if pts[4][0] > pts[6][0] and pts[4][0] > pts[10][0]  and  pts[4][1] < pts[18][1] and pts[4][1] < pts[14][1]:
                ch1 = 'N'
    
        if ch1 == 2:
            if distance(pts[12], pts[4]) > 42:
                ch1 = 'C'
            else:
                ch1 = 'O'
    
        if ch1 == 3:
            if (distance(pts[8], pts[12])) > 72:
                ch1 = 'G'
            else:
                ch1 = 'H'
    
        if ch1 == 7:
            if distance(pts[8], pts[4]) > 42:
                ch1 = 'Y'
            else:
                ch1 = 'J'
    
        if ch1 == 4:
            ch1 = 'L'
    
        if ch1 == 6:
            ch1 = 'X'
    
        if ch1 == 5:
            if pts[4][0] > pts[12][0] and pts[4][0] > pts[16][0] and pts[4][0] > pts[20][0]:
                if pts[8][1] < pts[5][1]:
                    ch1 = 'Z'
                else:
                    ch1 = 'Q'
            else:
                ch1 = 'P'
    
        if ch1 == 1:
            if (pts[6][1] > pts[8][1] and pts[10][1] > pts[12][1] and pts[14][1] > pts[16][1] and pts[18][1] >pts[20][1]):
                ch1 = 'B'
            if (pts[6][1] > pts[8][1] and pts[10][1] < pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] <pts[20][1]):
                ch1 = 'D'
            if (pts[6][1] < pts[8][1] and pts[10][1] > pts[12][1] and pts[14][1] > pts[16][1] and pts[18][1] > pts[20][1]):
                ch1 = 'F'
            if (pts[6][1] < pts[8][1] and pts[10][1] < pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] > pts[20][1]):
                ch1 = 'I'
            if (pts[6][1] > pts[8][1] and pts[10][1] > pts[12][1] and pts[14][1] > pts[16][1] and pts[18][1] < pts[20][1]):
                ch1 = 'W'
            if  (pts[6][1] > pts[8][1] and pts[10][1] > pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] < pts[20][1]) and pts[4][1]<pts[9][1]:
                ch1 = 'K'
            if ((distance(pts[8], pts[12]) - distance(pts[6], pts[10])) < 8) and (pts[6][1] > pts[8][1] and pts[10][1] > pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] < pts[20][1]):
                ch1 = 'U'
            if ((distance(pts[8], pts[12]) - distance(pts[6], pts[10])) >= 8) and (pts[6][1] > pts[8][1] and pts[10][1] > pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] < pts[20][1]) and (pts[4][1] >pts[9][1]):
                ch1 = 'V'
    
            if (pts[8][0] > pts[12][0]) and (pts[6][1] > pts[8][1] and pts[10][1] > pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] < pts[20][1]):
                ch1 = 'R'
    
        if ch1== 1 or 'E' or 'S' or 'X' or 'Y' or 'B':
            if (pts[6][1] > pts[8][1] and pts[10][1] < pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] > pts[20][1]):
                ch1 = 'Space'
    
        if ch1== 'E' or 'Y' or 'B':
            if (pts[4][0] < pts[5][0] ):
                ch1 = 'Next'

        if ch1== 'Next' or 'B' or 'C' or 'H' or 'F':
            if (pts[0][0] > pts[8][0] and pts[0][0] > pts[12][0] and pts[0][0] > pts[16][0] and pts[0][0] > pts[20][0]) and pts[4][1]<pts[8][1] and pts[4][1]<pts[12][1] and pts[4][1]<pts[16][1] and pts[4][1]<pts[20][1]:
                ch1 = 'Backspace'

        print("ch1=", ch1, " ch2=", ch2, " ch3=", ch3)
        kok.append(ch1)

                # # [0->aemnst][1->bfdiuvwkr][2->co][3->gh][4->l][5->pqz][6->x][7->yj]
        if ch1 != 1:
            if (ch1,ch2) in dicttt:
                dicttt[(ch1,ch2)] += 1
            else:
                dicttt[(ch1,ch2)] = 1

        frame = cv2.putText(frame, "Predicted " + str(ch1), (30, 80),
                                    cv2.FONT_HERSHEY_SIMPLEX,
                                    3, (0, 0, 255), 2, cv2.LINE_AA)


        # Add more conditions as needed...
        
        # Map integer predictions to letters
        letter_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H'}
        ch1 = letter_mapping.get(ch1, 'Unknown')

        print(f"Predicted characters: ch1={ch1}, ch2={ch2}, ch3={ch3}")

        if ch1 == 'Unknown':
            return

        if self.blank_flag == 0 and ch1 == 'A':  # Assuming 'A' represents blank in your case
            self.blank_flag = 1
        else:
            self.blank_flag = 0
            self.current_symbol = ch1
            self.ct[self.current_symbol] += 1
    
            if self.ct[self.current_symbol] > 20:
                self.ct = {char: 0 for char in ascii_uppercase}
                if self.current_symbol == self.prev_char:
                    self.count += 1
                else:
                    self.count = 0
                self.prev_char = self.current_symbol
    
                if self.count == 2:
                    self.count = 0
                    if self.current_symbol == 'Z':
                        self.str += " "
                    else:
                        self.str += self.current_symbol
        self.panel3.config(text=self.current_symbol, font=("Courier", 30, "bold"))
        self.panel5.config(text=self.str, font=("Courier", 30, "bold"))

    def action1(self):
        # Define what button1 should do
        pass

    def action2(self):
        # Define what button2 should do
        pass

    def action3(self):
        # Define what button3 should do
        pass

    def action4(self):
        # Define what button4 should do
        pass

    def speak_fun(self):
        self.speak_engine.say(self.str)
        self.speak_engine.runAndWait()

    def clear_fun(self):
        self.str = ""
        self.panel5.config(text=self.str, font=("Courier", 30, "bold"))

print("Starting Application...")
app = Application()
app.root.mainloop()


Starting Application...
Loading model...
Model loaded successfully.
1/1 [==============================] - 1s 1s/step
An error occurred: name 'distance' is not defined


Traceback (most recent call last):
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_17792\243799169.py", line 139, in video_loop
    self.predict(white_expanded, hand_landmarks.landmark)
  File "C:\Users\amish\AppData\Local\Temp\ipykernel_17792\243799169.py", line 296, in predict
    if ((distance(pts[8], pts[12]) - distance(pts[6], pts[10])) < 8) and (pts[6][1] > pts[8][1] and pts[10][1] > pts[12][1] and pts[14][1] < pts[16][1] and pts[18][1] < pts[20][1]):
NameError: name 'distance' is not defined. Did you mean: 'isinstance'?


Closing Application


In [ ]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import numpy as np
import traceback
from keras.models import load_model
from string import ascii_uppercase
import mediapipe as mp
import pyttsx3  # for text-to-speech

class Application:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Sign Language To Text Conversion")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("1300x700")

        # Initialize variables
        self.vs = cv2.VideoCapture(0)
        self.current_image = None
        self.current_image2 = None
        self.ccc = 0
        self.current_symbol = ""
        self.str = ""
        self.blank_flag = 0
        self.ct = {char: 0 for char in ascii_uppercase}
        self.prev_char = ''
        self.count = 0
        self.dicttt = {}

        # Load model
        print("Loading model...")
        self.model = load_model(r'A:\desktop\progit\Sign-Language-To-Text-and-Speech-Conversion\fine_tune_models\fintune2.h5')
        print("Model loaded successfully.")

        self.setup_gui()
        self.speak_engine = pyttsx3.init()

        self.video_loop()

    def setup_gui(self):
        self.panel = tk.Label(self.root)
        self.panel.place(x=100, y=3, width=480, height=640)

        self.panel2 = tk.Label(self.root)
        self.panel2.place(x=700, y=115, width=400, height=400)

        self.T = tk.Label(self.root)
        self.T.place(x=60, y=5)
        self.T.config(text="Sign Language To Text Conversion", font=("Courier", 30, "bold"))

        self.panel3 = tk.Label(self.root)  # Current Symbol
        self.panel3.place(x=280, y=585)

        self.T1 = tk.Label(self.root)
        self.T1.place(x=10, y=580)
        self.T1.config(text="Character :", font=("Courier", 30, "bold"))

        self.panel5 = tk.Label(self.root)  # Sentence
        self.panel5.place(x=260, y=632)

        self.T3 = tk.Label(self.root)
        self.T3.place(x=10, y=632)
        self.T3.config(text="Sentence :", font=("Courier", 30, "bold"))

        self.T4 = tk.Label(self.root)
        self.T4.place(x=10, y=700)
        self.T4.config(text="Suggestions :", fg="red", font=("Courier", 30, "bold"))

        self.b1 = tk.Button(self.root, command=self.action1)
        self.b1.place(x=390, y=700)

        self.b2 = tk.Button(self.root, command=self.action2)
        self.b2.place(x=590, y=700)

        self.b3 = tk.Button(self.root, command=self.action3)
        self.b3.place(x=790, y=700)

        self.b4 = tk.Button(self.root, command=self.action4)
        self.b4.place(x=990, y=700)

        self.speak = tk.Button(self.root, text="Speak", font=("Courier", 20), wraplength=100, command=self.speak_fun)
        self.speak.place(x=1305, y=630)

        self.clear = tk.Button(self.root, text="Clear", font=("Courier", 20), wraplength=100, command=self.clear_fun)
        self.clear.place(x=1205, y=630)

    def destructor(self):
        print("Closing Application")
        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows()

    def video_loop(self):
        mp_hands = mp.solutions.hands
        hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
        mp_drawing = mp.solutions.drawing_utils

        try:
            ret, frame = self.vs.read()
            if not ret:
                print("Failed to grab frame")
                self.root.after(10, self.video_loop)
                return

            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = hands.process(frame_rgb)

            if result.multi_hand_landmarks:
                for hand_landmarks in result.multi_hand_landmarks:
                    x_min, y_min = float('inf'), float('inf')
                    x_max, y_max = float('-inf'), float('-inf')

                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        x_min, y_min = min(x_min, x), min(y_min, y)
                        x_max, y_max = max(x_max, x), max(y_max, y)

                    bbox = (x_min, y_min, x_max - x_min, y_max - y_min)
                    image = frame[y_min:y_max, x_min:x_max]
                    white = np.ones((400, 400, 3), np.uint8) * 255

                    h, w, _ = image.shape
                    os_x = (400 - w) // 2
                    os_y = (400 - h) // 2

                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        cv2.circle(white, (x - x_min + os_x, y - y_min + os_y), 5, (0, 0, 255), -1)

                    mp_drawing.draw_landmarks(white, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    # Resize image to 128x128
                    white_resized = cv2.resize(white, (128, 128))
                    white_expanded = np.expand_dims(white_resized, axis=0)
                    white_expanded = np.array(white_expanded, dtype='float32')

                    self.predict(white_expanded, hand_landmarks.landmark)

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            self.panel.imgtk = imgtk
            self.panel.config(image=imgtk)

        except Exception as e:
            print(f"An error occurred: {e}")
            traceback.print_exc()

        self.root.after(10, self.video_loop)

    def distance(self, p1, p2):
        return np.linalg.norm(np.array(p1) - np.array(p2))

    def predict(self, test_image, landmarks):
        prob = np.array(self.model.predict(test_image)[0], dtype='float32')
        ch1 = np.argmax(prob, axis=0)
        prob[ch1] = 0
        ch2 = np.argmax(prob, axis=0)
        prob[ch2] = 0
        ch3 = np.argmax(prob, axis=0)
        prob[ch3] = 0

        pts = [(lm.x, lm.y) for lm in landmarks]
    
        pl = [ch1, ch2]

        # Example condition for adjustments (these need to be defined based on actual requirements)
        if pl == [5, 2] and pts[6][1] < pts[8][1]:
            ch1 = 0

        # Example for other specific conditions
        if pl == [2, 2] and pts[5][0] < pts[4][0]:
            ch1 = 0

        # Add more conditions as needed...
        
        # Map integer predictions to letters
        letter_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H'}
        ch1 = letter_mapping.get(ch1, 'Unknown')

        print(f"Predicted characters: ch1={ch1}, ch2={ch2}, ch3={ch3}")

        if ch1 == 'Unknown':
            return

        if self.blank_flag == 0 and ch1 == 'A':  # Assuming 'A' represents blank in your case
            self.blank_flag = 1
        else:
            self.blank_flag = 0
            self.current_symbol = ch1
            self.ct[self.current_symbol] += 1
    
            if self.ct[self.current_symbol] > 10:  # Reduce the time by decreasing the threshold
                self.ct = {char: 0 for char in ascii_uppercase}
                if self.current_symbol == self.prev_char:
                    self.count += 1
                else:
                    self.count = 0
                self.prev_char = self.current_symbol
    
                if self.count == 1:  # Reduce the time by decreasing the threshold
                    self.count = 0
                    if self.current_symbol == 'Z':
                        self.str += " "
                    else:
                        self.str += self.current_symbol
        self.panel3.config(text=self.current_symbol, font=("Courier", 30, "bold"))
        self.panel5.config(text=self.str, font=("Courier", 30, "bold"))

    def action1(self):
        # Define what button1 should do
        pass

    def action2(self):
        # Define what button2 should do
        pass

    def action3(self):
        # Define what button3 should do
        pass

    def action4(self):
        # Define what button4 should do
        pass

    def speak_fun(self):
        self.speak_engine.say(self.str)
        self.speak_engine.runAndWait()

    def clear_fun(self):
        self.str = ""
        self.panel5.config(text=self.str, font=("Courier", 30, "bold"))

print("Starting Application...")
app = Application()
app.root.mainloop()


Starting Application...
Loading model...
Model loaded successfully.
1/1 [==============================] - 0s 112ms/step
Predicted characters: ch1=D, ch2=1, ch3=0
1/1 [==============================] - 0s 45ms/step
Predicted characters: ch1=E, ch2=5, ch3=1
1/1 [==============================] - 0s 38ms/step
Predicted characters: ch1=B, ch2=4, ch3=0
1/1 [==============================] - 0s 42ms/step
Predicted characters: ch1=E, ch2=5, ch3=0
1/1 [==============================] - 0s 31ms/step
Predicted characters: ch1=E, ch2=5, ch3=0
1/1 [==============================] - 0s 32ms/step
Predicted characters: ch1=E, ch2=0, ch3=0
1/1 [==============================] - 0s 25ms/step
Predicted characters: ch1=E, ch2=0, ch3=0
1/1 [==============================] - 0s 32ms/step
Predicted characters: ch1=E, ch2=0, ch3=0
1/1 [==============================] - 0s 31ms/step
Predicted characters: ch1=E, ch2=0, ch3=0
1/1 [==============================] - 0s 36ms/step
Predicted characters: ch1=E, ch2=

In [2]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import numpy as np
import traceback
from keras.models import load_model
from string import ascii_uppercase
import mediapipe as mp
import pyttsx3  # for text-to-speech

class Application:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Sign Language To Text Conversion")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("1300x700")

        # Initialize variables
        self.vs = cv2.VideoCapture(0)
        self.current_image = None
        self.current_image2 = None
        self.ccc = 0
        self.current_symbol = ""
        self.str = ""
        self.blank_flag = 0
        self.ct = {char: 0 for char in ascii_uppercase}
        self.prev_char = ''
        self.count = 0
        self.dicttt = {}

        # Load model
        print("Loading model...")
        self.model = load_model(r'A:\desktop\progit\Sign-Language-To-Text-and-Speech-Conversion\fine_tune_models\fintune2.h5')
        print("Model loaded successfully.")

        self.setup_gui()
        self.speak_engine = pyttsx3.init()

        self.video_loop()

    def setup_gui(self):
        self.panel = tk.Label(self.root)
        self.panel.place(x=100, y=3, width=480, height=640)

        self.panel2 = tk.Label(self.root)
        self.panel2.place(x=700, y=115, width=400, height=400)

        self.T = tk.Label(self.root)
        self.T.place(x=60, y=5)
        self.T.config(text="Sign Language To Text Conversion", font=("Courier", 30, "bold"))

        self.panel3 = tk.Label(self.root)  # Current Symbol
        self.panel3.place(x=280, y=585)

        self.T1 = tk.Label(self.root)
        self.T1.place(x=10, y=580)
        self.T1.config(text="Character :", font=("Courier", 30, "bold"))

        self.panel5 = tk.Label(self.root)  # Sentence
        self.panel5.place(x=260, y=632)

        self.T3 = tk.Label(self.root)
        self.T3.place(x=10, y=632)
        self.T3.config(text="Sentence :", font=("Courier", 30, "bold"))

        self.T4 = tk.Label(self.root)
        self.T4.place(x=10, y=700)
        self.T4.config(text="Suggestions :", fg="red", font=("Courier", 30, "bold"))

        self.b1 = tk.Button(self.root, command=self.action1)
        self.b1.place(x=390, y=700)

        self.b2 = tk.Button(self.root, command=self.action2)
        self.b2.place(x=590, y=700)

        self.b3 = tk.Button(self.root, command=self.action3)
        self.b3.place(x=790, y=700)

        self.b4 = tk.Button(self.root, command=self.action4)
        self.b4.place(x=990, y=700)

        self.speak = tk.Button(self.root, text="Speak", font=("Courier", 20), wraplength=100, command=self.speak_fun)
        self.speak.place(x=1305, y=630)

        self.clear = tk.Button(self.root, text="Clear", font=("Courier", 20), wraplength=100, command=self.clear_fun)
        self.clear.place(x=1205, y=630)

    def destructor(self):
        print("Closing Application")
        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows()

    def video_loop(self):
        mp_hands = mp.solutions.hands
        hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
        mp_drawing = mp.solutions.drawing_utils

        try:
            ret, frame = self.vs.read()
            if not ret:
                print("Failed to grab frame")
                self.root.after(10, self.video_loop)
                return

            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = hands.process(frame_rgb)

            if result.multi_hand_landmarks:
                for hand_landmarks in result.multi_hand_landmarks:
                    x_min, y_min = float('inf'), float('inf')
                    x_max, y_max = float('-inf'), float('-inf')

                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        x_min, y_min = min(x_min, x), min(y_min, y)
                        x_max, y_max = max(x_max, x), max(y_max, y)

                    bbox = (x_min, y_min, x_max - x_min, y_max - y_min)
                    image = frame[y_min:y_max, x_min:x_max]
                    white = np.ones((400, 400, 3), np.uint8) * 255

                    h, w, _ = image.shape
                    os_x = (400 - w) // 2
                    os_y = (400 - h) // 2

                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        cv2.circle(white, (x - x_min + os_x, y - y_min + os_y), 5, (0, 0, 255), -1)

                    mp_drawing.draw_landmarks(white, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    # Resize image to 128x128
                    white_resized = cv2.resize(white, (128, 128))
                    white_expanded = np.expand_dims(white_resized, axis=0)
                    white_expanded = np.array(white_expanded, dtype='float32')

                    self.predict(white_expanded, hand_landmarks.landmark)

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            self.panel.imgtk = imgtk
            self.panel.config(image=imgtk)

        except Exception as e:
            print(f"An error occurred: {e}")
            traceback.print_exc()

        self.root.after(10, self.video_loop)

    def draw_hand_lines(self, white, os_x, os_y, pts):
        for t in range(0, 4):
            cv2.line(white, (pts[t][0] + os_x, pts[t][1] + os_y), 
                     (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), (0, 255, 0), 3)
        for t in range(5, 8):
            cv2.line(white, (pts[t][0] + os_x, pts[t][1] + os_y), 
                     (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), (0, 255, 0), 3)
        for t in range(9, 12):
            cv2.line(white, (pts[t][0] + os_x, pts[t][1] + os_y), 
                     (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), (0, 255, 0), 3)
        for t in range(13, 16):
            cv2.line(white, (pts[t][0] + os_x, pts[t][1] + os_y), 
                     (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), (0, 255, 0), 3)
        for t in range(17, 20):
            cv2.line(white, (pts[t][0] + os_x, pts[t][1] + os_y), 
                     (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), (0, 255, 0), 3)
        for t in [0, 5, 9, 13, 17]:
            cv2.line(white, (pts[t][0] + os_x, pts[t][1] + os_y), 
                     (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), (0, 255, 0), 3)
            cv2.line(white, (pts[t + 1][0] + os_x, pts[t + 1][1] + os_y), 
                     (pts[t + 2][0] + os_x, pts[t + 2][1] + os_y), (0, 255, 0), 3)
        for t in range(1, 20, 4):
            cv2.line(white, (pts[0][0] + os_x, pts[0][1] + os_y), 
                     (pts[t][0] + os_x, pts[t][1] + os_y), (0, 255, 0), 3)

    def distance(self, p1, p2):
        return np.linalg.norm(np.array(p1) - np.array(p2))

    def predict(self, test_image, landmarks):
        prob = np.array(self.model.predict(test_image)[0], dtype='float32')
        ch1 = np.argmax(prob, axis=0)
        prob[ch1] = 0
        ch2 = np.argmax(prob, axis=0)
        prob[ch2] = 0
        ch3 = np.argmax(prob, axis=0)
        prob[ch3] = 0

        pts = [(lm.x, lm.y) for lm in landmarks]
    
        pl = [ch1, ch2]

        # Example condition for adjustments (these need to be defined based on actual requirements)
        if pl == [5, 2] and pts[6][1] < pts[8][1]:
            ch1 = 0

        # Example for other specific conditions
        if pl == [2, 2] and pts[5][0] < pts[4][0]:
            ch1 = 0

        # Add more conditions as needed...
        
        # Map integer predictions to letters
        letter_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H'}
        ch1 = letter_mapping.get(ch1, 'Unknown')

        print(f"Predicted characters: ch1={ch1}, ch2={ch2}, ch3={ch3}")

        if ch1 == 'Unknown':
            return

        if self.blank_flag == 0 and ch1 == 'A':  # Assuming 'A' represents blank in your case
            self.blank_flag = 1
        else:
            self.blank_flag = 0
            self.current_symbol = ch1
            self.ct[self.current_symbol] += 1
    
            if self.ct[self.current_symbol] > 10:  # Reduce the time by decreasing the threshold
                self.ct = {char: 0 for char in ascii_uppercase}
                if self.current_symbol == self.prev_char:
                    self.count += 1
                else:
                    self.count = 0
                self.prev_char = self.current_symbol
    
                if self.count == 1:  # Reduce the time by decreasing the threshold
                    self.count = 0
                    if self.current_symbol == 'Z':
                        self.str += " "
                    else:
                        self.str += self.current_symbol
        self.panel3.config(text=self.current_symbol, font=("Courier", 30, "bold"))
        self.panel5.config(text=self.str, font=("Courier", 30, "bold"))

    def action1(self):
        # Define what button1 should do
        pass

    def action2(self):
        # Define what button2 should do
        pass

    def action3(self):
        # Define what button3 should do
        pass

    def action4(self):
        # Define what button4 should do
        pass

    def speak_fun(self):
        self.speak_engine.say(self.str)
        self.speak_engine.runAndWait()

    def clear_fun(self):
        self.str = ""
        self.panel5.config(text=self.str, font=("Courier", 30, "bold"))

print("Starting Application...")
app = Application()
app.root.mainloop()


Starting Application...
Loading model...
Model loaded successfully.
1/1 [==============================] - 1s 1s/step
Predicted characters: ch1=E, ch2=1, ch3=0
1/1 [==============================] - 0s 31ms/step
Predicted characters: ch1=B, ch2=14, ch3=0
1/1 [==============================] - 0s 16ms/step
Predicted characters: ch1=B, ch2=4, ch3=0
1/1 [==============================] - 0s 32ms/step
Predicted characters: ch1=B, ch2=0, ch3=0
1/1 [==============================] - 0s 16ms/step
Predicted characters: ch1=E, ch2=1, ch3=0
1/1 [==============================] - 0s 16ms/step
Predicted characters: ch1=B, ch2=4, ch3=14
1/1 [==============================] - 0s 32ms/step
Predicted characters: ch1=B, ch2=4, ch3=0
1/1 [==============================] - 0s 16ms/step
Predicted characters: ch1=Unknown, ch2=1, ch3=4
1/1 [==============================] - 0s 31ms/step
Predicted characters: ch1=B, ch2=4, ch3=0
1/1 [==============================] - 0s 24ms/step
Predicted characters: ch1=B,

In [4]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import numpy as np
import traceback
from keras.models import load_model
from string import ascii_uppercase
import mediapipe as mp
import pyttsx3  # for text-to-speech

class Application:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Sign Language To Text Conversion")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("1300x700")

        # Initialize variables
        self.vs = cv2.VideoCapture(0)
        self.current_image = None
        self.current_image2 = None
        self.ccc = 0
        self.current_symbol = ""
        self.str = ""
        self.blank_flag = 0
        self.ct = {char: 0 for char in ascii_uppercase}
        self.prev_char = ''
        self.count = 0
        self.dicttt = {}

        # Load model
        print("Loading model...")
        self.model = load_model(r'A:\desktop\progit\Sign-Language-To-Text-and-Speech-Conversion\fine_tune_models\fintune2.h5')
        print("Model loaded successfully.")

        self.setup_gui()
        self.speak_engine = pyttsx3.init()

        self.video_loop()

    def setup_gui(self):
        self.panel = tk.Label(self.root)
        self.panel.place(x=100, y=3, width=480, height=640)

        self.panel2 = tk.Label(self.root)
        self.panel2.place(x=700, y=115, width=400, height=400)

        self.T = tk.Label(self.root)
        self.T.place(x=60, y=5)
        self.T.config(text="Sign Language To Text Conversion", font=("Courier", 30, "bold"))

        self.panel3 = tk.Label(self.root)  # Current Symbol
        self.panel3.place(x=280, y=585)

        self.T1 = tk.Label(self.root)
        self.T1.place(x=10, y=580)
        self.T1.config(text="Character :", font=("Courier", 30, "bold"))

        self.panel5 = tk.Label(self.root)  # Sentence
        self.panel5.place(x=260, y=632)

        self.T3 = tk.Label(self.root)
        self.T3.place(x=10, y=632)
        self.T3.config(text="Sentence :", font=("Courier", 30, "bold"))

        self.T4 = tk.Label(self.root)
        self.T4.place(x=10, y=700)
        self.T4.config(text="Suggestions :", fg="red", font=("Courier", 30, "bold"))

        self.b1 = tk.Button(self.root, command=self.action1)
        self.b1.place(x=390, y=700)

        self.b2 = tk.Button(self.root, command=self.action2)
        self.b2.place(x=590, y=700)

        self.b3 = tk.Button(self.root, command=self.action3)
        self.b3.place(x=790, y=700)

        self.b4 = tk.Button(self.root, command=self.action4)
        self.b4.place(x=990, y=700)

        self.speak = tk.Button(self.root, text="Speak", font=("Courier", 20), wraplength=100, command=self.speak_fun)
        self.speak.place(x=1305, y=630)

        self.clear = tk.Button(self.root, text="Clear", font=("Courier", 20), wraplength=100, command=self.clear_fun)
        self.clear.place(x=1205, y=630)

    def destructor(self):
        print("Closing Application")
        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows()

    def video_loop(self):
        mp_hands = mp.solutions.hands
        hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
        mp_drawing = mp.solutions.drawing_utils

        try:
            ret, frame = self.vs.read()
            if not ret:
                print("Failed to grab frame")
                self.root.after(10, self.video_loop)
                return

            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = hands.process(frame_rgb)

            if result.multi_hand_landmarks:
                for hand_landmarks in result.multi_hand_landmarks:
                    x_min, y_min = float('inf'), float('inf')
                    x_max, y_max = float('-inf'), float('-inf')

                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        x_min, y_min = min(x_min, x), min(y_min, y)
                        x_max, y_max = max(x_max, x), max(y_max, y)

                    bbox = (x_min, y_min, x_max - x_min, y_max - y_min)
                    image = frame[y_min:y_max, x_min:x_max]
                    white = np.ones((400, 400, 3), np.uint8) * 255

                    h, w, _ = image.shape
                    os_x = (400 - w) // 2
                    os_y = (400 - h) // 2

                    pts = [(int(lm.x * 400), int(lm.y * 400)) for lm in hand_landmarks.landmark]
                    self.draw_hand_lines(white, pts)

                    mp_drawing.draw_landmarks(white, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    # Resize image to 128x128
                    white_resized = cv2.resize(white, (128, 128))
                    white_expanded = np.expand_dims(white_resized, axis=0)
                    white_expanded = np.array(white_expanded, dtype='float32')

                    self.predict(white_expanded, hand_landmarks.landmark)

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            self.panel.imgtk = imgtk
            self.panel.config(image=imgtk)

        except Exception as e:
            print(f"An error occurred: {e}")
            traceback.print_exc()

        self.root.after(10, self.video_loop)

    def draw_hand_lines(self, white, pts):
        connections = [
            (0, 1), (1, 2), (2, 3), (3, 4),  # Thumb
            (5, 6), (6, 7), (7, 8),  # Index
            (9, 10), (10, 11), (11, 12),  # Middle
            (13, 14), (14, 15), (15, 16),  # Ring
            (17, 18), (18, 19), (19, 20),  # Pinky
            (0, 5), (5, 9), (9, 13), (13, 17),  # Palm connections
            (0, 17)  # Palm base connection
        ]

        for start, end in connections:
            pt1 = pts[start]
            pt2 = pts[end]
            cv2.line(white, pt1, pt2, (0, 255, 0), 3)
            print(f"Drawing line: {pt1} to {pt2}")

    def distance(self, p1, p2):
        return np.linalg.norm(np.array(p1) - np.array(p2))

    def predict(self, test_image, landmarks):
        prob = np.array(self.model.predict(test_image)[0], dtype='float32')
        ch1 = np.argmax(prob, axis=0)
        prob[ch1] = 0
        ch2 = np.argmax(prob, axis=0)
        prob[ch2] = 0
        ch3 = np.argmax(prob, axis=0)
        prob[ch3] = 0

        pts = [(lm.x, lm.y) for lm in landmarks]
    
        pl = [ch1, ch2]

        # Example condition for adjustments (these need to be defined based on actual requirements)
        if pl == [5, 2] and pts[6][1] < pts[8][1]:
            ch1 = 0

        # Example for other specific conditions
        if pl == [2, 2] and pts[5][0] < pts[4][0]:
            ch1 = 0

        # Add more conditions as needed...
        
        # Map integer predictions to letters
        letter_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H'}
        ch1 = letter_mapping.get(ch1, 'Unknown')

        print(f"Predicted characters: ch1={ch1}, ch2={ch2}, ch3={ch3}")

        if ch1 == 'Unknown':
            return

        if self.blank_flag == 0 and ch1 == 'A':  # Assuming 'A' represents blank in your case
            self.blank_flag = 1
        else:
            self.blank_flag = 0
            self.current_symbol = ch1
            self.ct[self.current_symbol] += 1
    
            if self.ct[self.current_symbol] > 10:  # Reduce the time by decreasing the threshold
                self.ct = {char: 0 for char in ascii_uppercase}
                if self.current_symbol == self.prev_char:
                    self.count += 1
                else:
                    self.count = 0
                self.prev_char = self.current_symbol
    
                if self.count == 1:  # Reduce the time by decreasing the threshold
                    self.count = 0
                    if self.current_symbol == 'Z':
                        self.str += " "
                    else:
                        self.str += self.current_symbol
        self.panel3.config(text=self.current_symbol, font=("Courier", 30, "bold"))
        self.panel5.config(text=self.str, font=("Courier", 30, "bold"))

    def action1(self):
        # Define what button1 should do
        pass

    def action2(self):
        # Define what button2 should do
        pass

    def action3(self):
        # Define what button3 should do
        pass

    def action4(self):
        # Define what button4 should do
        pass

    def speak_fun(self):
        self.speak_engine.say(self.str)
        self.speak_engine.runAndWait()

    def clear_fun(self):
        self.str = ""
        self.panel5.config(text=self.str, font=("Courier", 30, "bold"))

print("Starting Application...")
app = Application()
app.root.mainloop()


Starting Application...
Loading model...
Model loaded successfully.
Drawing line: (270, 383) to (242, 368)
Drawing line: (242, 368) to (220, 334)
Drawing line: (220, 334) to (209, 304)
Drawing line: (209, 304) to (202, 278)
Drawing line: (242, 269) to (233, 218)
Drawing line: (233, 218) to (227, 185)
Drawing line: (227, 185) to (224, 157)
Drawing line: (266, 266) to (264, 211)
Drawing line: (264, 211) to (261, 176)
Drawing line: (261, 176) to (260, 145)
Drawing line: (289, 277) to (291, 224)
Drawing line: (291, 224) to (292, 190)
Drawing line: (292, 190) to (293, 161)
Drawing line: (311, 298) to (327, 263)
Drawing line: (327, 263) to (338, 240)
Drawing line: (338, 240) to (348, 219)
Drawing line: (270, 383) to (242, 269)
Drawing line: (242, 269) to (266, 266)
Drawing line: (266, 266) to (289, 277)
Drawing line: (289, 277) to (311, 298)
Drawing line: (270, 383) to (311, 298)
1/1 [==============================] - 0s 144ms/step
Predicted characters: ch1=C, ch2=0, ch3=0
Drawing line: (262

In [ ]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import numpy as np
import traceback
from keras.models import load_model
from string import ascii_uppercase
import mediapipe as mp
import pyttsx3  # for text-to-speech

class Application:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Sign Language To Text Conversion")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("1300x700")

        # Initialize variables
        self.vs = cv2.VideoCapture(0)
        self.current_image = None
        self.current_image2 = None
        self.ccc = 0
        self.current_symbol = ""
        self.str = ""
        self.blank_flag = 0
        self.ct = {char: 0 for char in ascii_uppercase}
        self.prev_char = ''
        self.count = 0
        self.dicttt = {}

        # Load model
        print("Loading model...")
        self.model = load_model(r'A:\desktop\progit\Sign-Language-To-Text-and-Speech-Conversion\fine_tune_models\fintune2.h5')
        print("Model loaded successfully.")

        self.setup_gui()
        self.speak_engine = pyttsx3.init()

        self.video_loop()

    def setup_gui(self):
        self.panel = tk.Label(self.root)
        self.panel.place(x=100, y=3, width=480, height=640)

        self.panel2 = tk.Label(self.root)
        self.panel2.place(x=700, y=115, width=400, height=400)

        self.T = tk.Label(self.root)
        self.T.place(x=60, y=5)
        self.T.config(text="Sign Language To Text Conversion", font=("Courier", 30, "bold"))

        self.panel3 = tk.Label(self.root)  # Current Symbol
        self.panel3.place(x=280, y=585)

        self.T1 = tk.Label(self.root)
        self.T1.place(x=10, y=580)
        self.T1.config(text="Character :", font=("Courier", 30, "bold"))

        self.panel5 = tk.Label(self.root)  # Sentence
        self.panel5.place(x=260, y=632)

        self.T3 = tk.Label(self.root)
        self.T3.place(x=10, y=632)
        self.T3.config(text="Sentence :", font=("Courier", 30, "bold"))

        self.T4 = tk.Label(self.root)
        self.T4.place(x=10, y=700)
        self.T4.config(text="Suggestions :", fg="red", font=("Courier", 30, "bold"))

        self.b1 = tk.Button(self.root, command=self.action1)
        self.b1.place(x=390, y=700)

        self.b2 = tk.Button(self.root, command=self.action2)
        self.b2.place(x=590, y=700)

        self.b3 = tk.Button(self.root, command=self.action3)
        self.b3.place(x=790, y=700)

        self.b4 = tk.Button(self.root, command=self.action4)
        self.b4.place(x=990, y=700)

        self.speak = tk.Button(self.root, text="Speak", font=("Courier", 20), wraplength=100, command=self.speak_fun)
        self.speak.place(x=1305, y=630)

        self.clear = tk.Button(self.root, text="Clear", font=("Courier", 20), wraplength=100, command=self.clear_fun)
        self.clear.place(x=1205, y=630)

    def destructor(self):
        print("Closing Application")
        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows()

    def video_loop(self):
        mp_hands = mp.solutions.hands
        hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
        mp_drawing = mp.solutions.drawing_utils

        try:
            ret, frame = self.vs.read()
            if not ret:
                print("Failed to grab frame")
                self.root.after(10, self.video_loop)
                return

            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = hands.process(frame_rgb)

            if result.multi_hand_landmarks:
                for hand_landmarks in result.multi_hand_landmarks:
                    x_min, y_min = float('inf'), float('inf')
                    x_max, y_max = float('-inf'), float('-inf')

                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        x_min, y_min = min(x_min, x), min(y_min, y)
                        x_max, y_max = max(x_max, x), max(y_max, y)

                    bbox = (x_min, y_min, x_max - x_min, y_max - y_min)
                    image = frame[y_min:y_max, x_min:x_max]
                    white = np.ones((400, 400, 3), np.uint8) * 255

                    h, w, _ = image.shape
                    os_x = (400 - w) // 2
                    os_y = (400 - h) // 2

                    pts = [(int(lm.x * 400), int(lm.y * 400)) for lm in hand_landmarks.landmark]
                    self.draw_hand_lines(white, pts)

                    mp_drawing.draw_landmarks(white, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    # Resize image to 128x128
                    white_resized = cv2.resize(white, (128, 128))
                    white_expanded = np.expand_dims(white_resized, axis=0)
                    white_expanded = np.array(white_expanded, dtype='float32')

                    self.predict(white_expanded, hand_landmarks.landmark)

                    # Display the white image with drawn hand landmarks in panel2
                    white_img = Image.fromarray(white)
                    white_imgtk = ImageTk.PhotoImage(image=white_img)
                    self.panel2.imgtk = white_imgtk
                    self.panel2.config(image=white_imgtk)

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            self.panel.imgtk = imgtk
            self.panel.config(image=imgtk)

        except Exception as e:
            print(f"An error occurred: {e}")
            traceback.print_exc()

        self.root.after(10, self.video_loop)

    def draw_hand_lines(self, white, pts):
        connections = [
            (0, 1), (1, 2), (2, 3), (3, 4),  # Thumb
            (5, 6), (6, 7), (7, 8),  # Index
            (9, 10), (10, 11), (11, 12),  # Middle
            (13, 14), (14, 15), (15, 16),  # Ring
            (17, 18), (18, 19), (19, 20),  # Pinky
            (0, 5), (5, 9), (9, 13), (13, 17),  # Palm connections
            (0, 17)  # Palm base connection
        ]

        for start, end in connections:
            pt1 = pts[start]
            pt2 = pts[end]
            cv2.line(white, pt1, pt2, (0, 255, 0), 3)
            print(f"Drawing line: {pt1} to {pt2}")

    def distance(self, p1, p2):
        return np.linalg.norm(np.array(p1) - np.array(p2))

    def predict(self, test_image, landmarks):
        prob = np.array(self.model.predict(test_image)[0], dtype='float32')
        ch1 = np.argmax(prob, axis=0)
        prob[ch1] = 0
        ch2 = np.argmax(prob, axis=0)
        prob[ch2] = 0
        ch3 = np.argmax(prob, axis=0)
        prob[ch3] = 0

        pts = [(lm.x, lm.y) for lm in landmarks]
    
        pl = [ch1, ch2]

        # Example condition for adjustments (these need to be defined based on actual requirements)
        if pl == [5, 2] and pts[6][1] < pts[8][1]:
            ch1 = 0

        # Example for other specific conditions
        if pl == [2, 2] and pts[5][0] < pts[4][0]:
            ch1 = 0

        # Add more conditions as needed...
        
        # Map integer predictions to letters
        letter_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H'}
        ch1 = letter_mapping.get(ch1, 'Unknown')

        print(f"Predicted characters: ch1={ch1}, ch2={ch2}, ch3={ch3}")

        if ch1 == 'Unknown':
            return

        if self.blank_flag == 0 and ch1 == 'A':  # Assuming 'A' represents blank in your case
            self.blank_flag = 1
        else:
            self.blank_flag = 0
            self.current_symbol = ch1
            self.ct[self.current_symbol] += 1
    
            if self.ct[self.current_symbol] > 10:  # Reduce the time by decreasing the threshold
                self.ct = {char: 0 for char in ascii_uppercase}
                if self.current_symbol == self.prev_char:
                    self.count += 1
                else:
                    self.count = 0
                self.prev_char = self.current_symbol
    
                if self.count == 1:  # Reduce the time by decreasing the threshold
                    self.count = 0
                    if self.current_symbol == 'Z':
                        self.str += " "
                    else:
                        self.str += self.current_symbol
        self.panel3.config(text=self.current_symbol, font=("Courier", 30, "bold"))
        self.panel5.config(text=self.str, font=("Courier", 30, "bold"))

    def action1(self):
        # Define what button1 should do
        pass

    def action2(self):
        # Define what button2 should do
        pass

    def action3(self):
        # Define what button3 should do
        pass

    def action4(self):
        # Define what button4 should do
        pass

    def speak_fun(self):
        self.speak_engine.say(self.str)
        self.speak_engine.runAndWait()

    def clear_fun(self):
        self.str = ""
        self.panel5.config(text=self.str, font=("Courier", 30, "bold"))

print("Starting Application...")
app = Application()
app.root.mainloop()


In [1]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
import numpy as np
import traceback
from keras.models import load_model
from string import ascii_uppercase
import mediapipe as mp
import pyttsx3  # for text-to-speech

class Application:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Sign Language To Text Conversion")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("1300x700")

        # Initialize variables
        self.vs = cv2.VideoCapture(0)
        self.current_image = None
        self.current_image2 = None
        self.ccc = 0
        self.current_symbol = ""
        self.str = ""
        self.blank_flag = 0
        self.ct = {char: 0 for char in ascii_uppercase}
        self.prev_char = ''
        self.count = 0
        self.dicttt = {}

        # Load model
        print("Loading model...")
        self.model = load_model(r'A:\desktop\progit\Sign-Language-To-Text-and-Speech-Conversion\fine_tune_models\fintune2.h5')
        print("Model loaded successfully.")

        self.setup_gui()
        self.speak_engine = pyttsx3.init()

        self.video_loop()

    def setup_gui(self):
        self.panel = tk.Label(self.root)
        self.panel.place(x=100, y=3, width=480, height=640)

        self.panel2 = tk.Label(self.root)
        self.panel2.place(x=700, y=115, width=400, height=400)

        self.T = tk.Label(self.root)
        self.T.place(x=60, y=5)
        self.T.config(text="Sign Language To Text Conversion", font=("Courier", 30, "bold"))

        self.panel3 = tk.Label(self.root)  # Current Symbol
        self.panel3.place(x=280, y=585)

        self.T1 = tk.Label(self.root)
        self.T1.place(x=10, y=580)
        self.T1.config(text="Character :", font=("Courier", 30, "bold"))

        self.panel5 = tk.Label(self.root)  # Sentence
        self.panel5.place(x=260, y=632)

        self.T3 = tk.Label(self.root)
        self.T3.place(x=10, y=632)
        self.T3.config(text="Sentence :", font=("Courier", 30, "bold"))

        self.T4 = tk.Label(self.root)
        self.T4.place(x=10, y=700)
        self.T4.config(text="Suggestions :", fg="red", font=("Courier", 30, "bold"))

        self.b1 = tk.Button(self.root, command=self.action1)
        self.b1.place(x=390, y=700)

        self.b2 = tk.Button(self.root, command=self.action2)
        self.b2.place(x=590, y=700)

        self.b3 = tk.Button(self.root, command=self.action3)
        self.b3.place(x=790, y=700)

        self.b4 = tk.Button(self.root, command=self.action4)
        self.b4.place(x=990, y=700)

        self.speak = tk.Button(self.root, text="Speak", font=("Courier", 20), wraplength=100, command=self.speak_fun)
        self.speak.place(x=1305, y=630)

        self.clear = tk.Button(self.root, text="Clear", font=("Courier", 20), wraplength=100, command=self.clear_fun)
        self.clear.place(x=1205, y=630)

    def destructor(self):
        print("Closing Application")
        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows()

    def video_loop(self):
        mp_hands = mp.solutions.hands
        hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
        mp_drawing = mp.solutions.drawing_utils

        try:
            ret, frame = self.vs.read()
            if not ret:
                print("Failed to grab frame")
                self.root.after(10, self.video_loop)
                return

            frame = cv2.flip(frame, 1)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            result = hands.process(frame_rgb)

            if result.multi_hand_landmarks:
                for hand_landmarks in result.multi_hand_landmarks:
                    x_min, y_min = float('inf'), float('inf')
                    x_max, y_max = float('-inf'), float('-inf')

                    for lm in hand_landmarks.landmark:
                        x, y = int(lm.x * frame.shape[1]), int(lm.y * frame.shape[0])
                        x_min, y_min = min(x_min, x), min(y_min, y)
                        x_max, y_max = max(x_max, x), max(y_max, y)

                    bbox = (x_min, y_min, x_max - x_min, y_max - y_min)
                    image = frame[y_min:y_max, x_min:x_max]
                    white = np.ones((400, 400, 3), np.uint8) * 255

                    h, w, _ = image.shape
                    os_x = (400 - w) // 2
                    os_y = (400 - h) // 2

                    pts = [(int(lm.x * 400), int(lm.y * 400)) for lm in hand_landmarks.landmark]
                    self.draw_hand_lines(white, pts)

                    mp_drawing.draw_landmarks(white, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    # Resize image to 128x128
                    white_resized = cv2.resize(white, (128, 128))
                    white_expanded = np.expand_dims(white_resized, axis=0)
                    white_expanded = np.array(white_expanded, dtype='float32')

                    self.predict(white_expanded, hand_landmarks.landmark)

                    # Display the white image with drawn hand landmarks in panel2
                    white_img = Image.fromarray(white)
                    white_imgtk = ImageTk.PhotoImage(image=white_img)
                    self.panel2.imgtk = white_imgtk
                    self.panel2.config(image=white_imgtk)

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            self.panel.imgtk = imgtk
            self.panel.config(image=imgtk)

        except Exception as e:
            print(f"An error occurred: {e}")
            traceback.print_exc()

        self.root.after(10, self.video_loop)

    def draw_hand_lines(self, white, pts):
        connections = [
            (0, 1), (1, 2), (2, 3), (3, 4),  # Thumb
            (5, 6), (6, 7), (7, 8),  # Index
            (9, 10), (10, 11), (11, 12),  # Middle
            (13, 14), (14, 15), (15, 16),  # Ring
            (17, 18), (18, 19), (19, 20),  # Pinky
            (0, 5), (5, 9), (9, 13), (13, 17),  # Palm connections
            (0, 17)  # Palm base connection
        ]

        for start, end in connections:
            pt1 = pts[start]
            pt2 = pts[end]
            cv2.line(white, pt1, pt2, (0, 255, 0), 3)
            print(f"Drawing line: {pt1} to {pt2}")

    def distance(self, p1, p2):
        return np.linalg.norm(np.array(p1) - np.array(p2))

    def predict(self, test_image, landmarks):
        prob = np.array(self.model.predict(test_image)[0], dtype='float32')
        ch1 = np.argmax(prob, axis=0)
        prob[ch1] = 0
        ch2 = np.argmax(prob, axis=0)
        prob[ch2] = 0
        ch3 = np.argmax(prob, axis=0)
        prob[ch3] = 0

        pts = [(lm.x, lm.y) for lm in landmarks]
    
        pl = [ch1, ch2]

        # Example condition for adjustments (these need to be defined based on actual requirements)
        if pl == [5, 2] and pts[6][1] < pts[8][1]:
            ch1 = 0

        # Example for other specific conditions
        if pl == [2, 2] and pts[5][0] < pts[4][0]:
            ch1 = 0

        # Add more conditions as needed...
        
        # Map integer predictions to letters
        letter_mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H'}
        ch1 = letter_mapping.get(ch1, 'Unknown')

        print(f"Predicted characters: ch1={ch1}, ch2={ch2}, ch3={ch3}")

        if ch1 == 'Unknown':
            return

        if self.blank_flag == 0 and ch1 == 'A':  # Assuming 'A' represents blank in your case
            self.blank_flag = 1
        else:
            self.blank_flag = 0
            self.current_symbol = ch1
            self.ct[self.current_symbol] += 1
    
            if self.ct[self.current_symbol] > 10:  # Reduce the time by decreasing the threshold
                self.ct = {char: 0 for char in ascii_uppercase}
                if self.current_symbol == self.prev_char:
                    self.count += 1
                else:
                    self.count = 0
                self.prev_char = self.current_symbol
    
                if self.count == 1:  # Reduce the time by decreasing the threshold
                    self.count = 0
                    if self.current_symbol == 'Z':
                        self.str += " "
                    else:
                        self.str += self.current_symbol
        self.panel3.config(text=self.current_symbol, font=("Courier", 30, "bold"))
        self.panel5.config(text=self.str, font=("Courier", 30, "bold"))

    def action1(self):
        # Define what button1 should do
        pass

    def action2(self):
        # Define what button2 should do
        pass

    def action3(self):
        # Define what button3 should do
        pass

    def action4(self):
        # Define what button4 should do
        pass

    def speak_fun(self):
        self.speak_engine.say(self.str)
        self.speak_engine.runAndWait()

    def clear_fun(self):
        self.str = ""
        self.panel5.config(text=self.str, font=("Courier", 30, "bold"))

print("Starting Application...")
app = Application()
app.root.mainloop()


    

Starting Application...
Loading model...
Model loaded successfully.
Drawing line: (110, 322) to (87, 309)
Drawing line: (87, 309) to (66, 289)
Drawing line: (66, 289) to (56, 268)
Drawing line: (56, 268) to (53, 251)
Drawing line: (47, 293) to (43, 249)
Drawing line: (43, 249) to (57, 240)
Drawing line: (57, 240) to (65, 247)
Drawing line: (56, 299) to (55, 244)
Drawing line: (55, 244) to (72, 239)
Drawing line: (72, 239) to (78, 250)
Drawing line: (71, 301) to (72, 244)
Drawing line: (72, 244) to (89, 243)
Drawing line: (89, 243) to (94, 256)
Drawing line: (89, 302) to (95, 253)
Drawing line: (95, 253) to (105, 253)
Drawing line: (105, 253) to (108, 264)
Drawing line: (110, 322) to (47, 293)
Drawing line: (47, 293) to (56, 299)
Drawing line: (56, 299) to (71, 301)
Drawing line: (71, 301) to (89, 302)
Drawing line: (110, 322) to (89, 302)
1/1 [==============================] - 1s 886ms/step
Predicted characters: ch1=G, ch2=0, ch3=0
Drawing line: (110, 330) to (90, 310)
Drawing line: (9